**Measuring Demand Forecasting benefits series**

# Generating forecasts with Amazon SageMaker Canvas

> *This notebook should work with the **`Data Science 3.0`** kernel in SageMaker Studio (older versions may see errors)*
>
> ⚠️ ***If** running the optional data preparation section, you'll need to switch to a larger instance size such as `ml.m5.large` (2 vCPU + 8 GiB RAM). Otherwise, any instance type should be fine.*

In this notebook we'll use [Amazon SageMaker Canvas](https://aws.amazon.com/sagemaker/canvas/) to build and export a forecast through the no-code, business analyst-oriented Canvas UI.

> ⚠️ **A note on cost:** At the time of writing, this example's prepared dataset is approximately 27M cells which, per [the pricing page](https://aws.amazon.com/sagemaker/canvas/pricing/), may cost upwards of $500 to train a model in Canvas. If you're just exploring at this stage and have no strong need for a business-user-friendly UI, you could consider the [Amazon Forecast notebook](1.3.%20Run%20Amazon%20Forecast.ipynb) instead for a lower-cost option.

## Contents

1. [Canvas pre-requisites](#prereqs)
1. [(Optional) Data preparation](#dataprep)
    - [Dependencies and setup](#Dependencies-and-setup)
    - [Target Time-Series (TTS)](#tts)
    - [Related Time-Series (RTS)](#rts)
    - [The combined SageMaker Canvas dataset](#The-combined-SageMaker-Canvas-dataset)
1. [Import your dataset to SageMaker Canvas](#Import-your-dataset-to-SageMaker-Canvas)
1. [Configure and train your model](#Configure-and-train-your-model)
1. [Review and export forecast results](#Review-and-export-forecast-results)
1. [Next steps](#Next-steps)

## Canvas pre-requisites<a class="anchor" id="prereqs"></a>

To follow the instructions below, you'll need SageMaker Canvas set up: See the ["getting started"](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas-getting-started.html#canvas-prerequisites) and ["setting up"](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas-setting-up.html) sections of the developer guide for steps.

In particular, you'll also need to [enable time-series forecasting](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas-set-up-forecast.html) for your Canvas user(s).

If skipping the optional data preparation section below, you'll also need to [enable upload of local files](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas-set-up-local-upload.html) through the Canvas UI.

## (Optional) Data preparation<a class="anchor" id="dataprep"></a>

> ℹ️ **You don't need to run any of the code in this section.**
>
> We've already prepared a Canvas-ready file for the sample dataset and made it available for [download here](https://measuring-forecast-benefits-assets.s3.amazonaws.com/dataset/v1/benefits_demo_canvas.csv). This section is provided to illustrate how the file was generated, in case you'd like to re-use elements of the process for your own custom datasets.

To build a forecast with SageMaker Canvas, we need to prepare our dataset in line with the [format requirements documented in the developer guide](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas-time-series.html).

To be specific, we'll want a CSV in "flat file" format (date as one column, not in 2D "pivot table" style) where our target value finishes at the end of the historical period, but any additional input variables continue on to cover the forecast period too - similar to the format shown below:

| **timestamp** | **item_id** | **location** | **demand** | **weekend_hol_flag** | **promo** | **unit_price** |
|:-------------:|:-----------:|:------------:|-----------:|---------------------:|----------:|---------------:|
| 2017-01-01 | Hoodie_gray_L | Brazil_Store1 | 12 | 2 | 1.0 | 24.20 |
| 2017-01-02 | Hoodie_gray_L | Brazil_Store1 | 0 | 0 | 1.0 | 24.20 |
| ... | ... | ... | ... | ... | ... | ... |
| 2019-11-30 | Hoodie_gray_L | Brazil_Store1 | 4 | 0 | 1.0 | 24.20 |
| **2019-12-01** | Hoodie_gray_L | Brazil_Store1 |  | 0 | 1.0 | 24.20 |
| **...** | ... | ... | ... | ... | ... | ... |
| **2019-12-30** | Hoodie_gray_L | Brazil_Store1 |  | 0 | 1.0 | 24.20 |
| **2019-12-31** | Hoodie_gray_L | Brazil_Store1 |  | 0 | 1.0 | 24.20 |

Note that in our sample dataset, historical actual sales finish on 2019-12-31. Other notebooks in this series forecast for January 2020 but simultaneously collect evaluation data for December 2019, but SageMaker Canvas doesn't (at time of writing) have an equivalent for Amazon Forecast's "backtest export" feature.

In this notebook we'll use Python to prepare this file from the source data for consistency with the rest of the series - but you could use other no-code data tools if you prefer.

### Dependencies and setup

First we'll import the libraries this notebook needs, and configure the forecast horizon and Amazon S3 data location.

To be able to compare our "forecast" with actuals and the (monthly) baseline rolling average model, we'll actually be predicting the final month covered by the dataset: December 2019.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os
from datetime import datetime

# External Dependencies:
import pandas as pd  # Tabular/dataframe processing tools
import sagemaker  # SageMaker SDK used just to look up default S3 bucket

# Local Dependencies:
import util

# Configuration:
BUCKET_NAME = sagemaker.Session().default_bucket()
BUCKET_PREFIX = "measuring-forecast-benefits/"

os.makedirs("dataset", exist_ok=True)

HISTORY_START = datetime(year=2017, month=1, day=1)
FORECAST_START = datetime(year=2019, month=12, day=1)

FORECAST_HORIZON = pd.offsets.Day() * 31  # or e.g. Hour(), Week(), MonthEnd()
print(f"Configured forecast horizon: {FORECAST_HORIZON}")
FORECAST_FREQ = FORECAST_HORIZON.base.freqstr  # This should be Amazon Forecast compatible
print(f"({FORECAST_HORIZON.n} units of '{FORECAST_FREQ}')\n")

### Target Time-Series (TTS)<a class="anchor" id="tts"></a>

The thing we want to predict is called the "target": In this case, the number of units we can sell of each item or the "demand". Historical sales data will provide this:

In [ ]:
tts_df = util.analytics.filter_to_period(
    pd.read_parquet("s3://measuring-forecast-benefits-assets/dataset/v1/sales.parquet"),
    period_start=HISTORY_START,
    period_end=FORECAST_START,  # (Exclusive)
    timestamp_col_name='date',
)

tts_df

This dataset is already in line with our [target format](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas-time-series.html), so we'll simply *index* it by the key columns (timestamp, SKU, location) to enable joining with other variables later:

In [ ]:
FORECAST_DIMENSIONS = [c for c in tts_df.columns if c not in ("date", "sales", "sku")]
print(f"Forecast Dimensions:\n  {FORECAST_DIMENSIONS}")

tts_df = tts_df.set_index(["date", "sku"] + FORECAST_DIMENSIONS)
tts_df

### Related Time-Series (RTS)<a class="anchor" id="rts"></a>

We can include other input signals that **vary over time** to help the model predict the target, which for consistency with Amazon Forecast we'll call "related time-series". Popular time-varying features to help predict future demand include pricing and promotions, public holidays and events, and even weather information.

In this sample we'll build related time-series from two base datasets: Public holidays by country, and product prices/promotions. To use them with SageMaker Canvas, we'll need to consolidate them together into the same data file together with the target (TTS) from earlier.

To understand *which* reference data we need, we'll first extract the unique list of all item+location combinations in the scope of the dataset, and map those to countries (for public holidays) and product types (for price data):

In [ ]:
item_location_combos = tts_df.reset_index()[["sku", "location"]].drop_duplicates()
item_location_combos["product"] = item_location_combos["sku"].str.split("_").str[0]
item_location_combos["country"] = item_location_combos["location"].str.split("_").str[0]
item_location_combos

#### Weekends and holidays

The source weekend and holiday data has already been prepared in a flat file format. However, it extends for a full year beyond our TTS end date - so we need to trim it for only the forecasting period of interest:

In [ ]:
holiday_raw_df = pd.read_csv(
    "s3://measuring-forecast-benefits-assets/dataset/v1/weekend_holiday_flag.csv",
)
holiday_raw_df["date"] = pd.to_datetime(holiday_raw_df["date"])  # (As CSV)

# Filter out any data beyond the end of the forecasting horizon:
holiday_raw_df = util.analytics.filter_to_period(
    holiday_raw_df,
    period_start=HISTORY_START,
    period_end=FORECAST_START + FORECAST_HORIZON,
    timestamp_col_name="date",
)

holiday_raw_df

#### Prices and promotions

The price and promotion data is likewise available in a flat file format already, but needs some extra processing:

Typically, teams will have historical price data from one source and need to add in additional data to *project forward* what expected pricing will be over the forecast period - perhaps even modelling multiple scenarios with different forecast models.

That's not actually the case here (because we're building a retrospective "forecast" for a period where we already have historical data), but we'll simulate it by filtering out the final month of historical data:

> ⚠️ **Note:** This is not quite aligned with the [Amazon Forecast notebook](1.3.%20Run%20Amazon%20Forecast.ipynb), where we forecast 2020-01 (with projected prices) and back-test 2019-12 (with actual prices) - which should slightly disadvantage SageMaker Canvas in the final comparison. You could instead adjust the `period_end` filter here and skip the future projection below if you want a more direct comparison. Since SageMaker Canvas forecasting *uses Amazon Forecast under the hood anyway* though, we felt it was more important to show a realistic workflow than focus on comparing between the two.

In [ ]:
# Load the real/historical pricing:
prices_raw_df = util.analytics.filter_to_period(
    pd.read_parquet("s3://measuring-forecast-benefits-assets/dataset/v1/prices_promos.parquet"),
    period_start=HISTORY_START,
    period_end=FORECAST_START,  # (Omitting `+ FORECAST_HORIZON` as noted above)
    timestamp_col_name="date",
)
prices_raw_df

To project final prices forward, we'll first create a dataframe of empty (`NaN`) placeholders for all the future dates and products/countries:

In [ ]:
prices_dimensions = [
    c for c in prices_raw_df.columns if c not in ("date", "promo", "unit_price")
]

prices_future = pd.merge(
    # Range of dates in the forecast period:
    pd.date_range(
        FORECAST_START,
        FORECAST_START + FORECAST_HORIZON,
        freq=FORECAST_FREQ,
        inclusive="left",
        name="date",
    ).to_series(),
    # Unique combinations of country+product:
    prices_raw_df[prices_dimensions].drop_duplicates(),
    # Cross join (all combinations):
    how="cross",
)
prices_future["promo"] = float("nan")
prices_future["unit_price"] = float("nan")
prices_future

Then we can combine the past dataset and future placeholders to fill forward each product's final historical price across the full forecast window:

In [ ]:
prices_projected_df = pd.concat(
    # Concatenate the future placeholders with the historical prices
    [prices_raw_df, prices_future]
).set_index(
    # Index by key dimensions *before* date...
    prices_dimensions + ["date"]
).sort_index().groupby(
    # ...so we can sort by product/SKU first, and then ffill() any gaps
    level=prices_dimensions
).ffill().reset_index()

prices_projected_df

If you like, you can inspect this DataFrame to validate the continuity (i.e. `Brazil` `Gloves` will keep using the same `promo` and `unit_price` for records after `TTS_PERIOD_END` - and likewise for each other combination of country and product type).

> ℹ️ If you instead loaded actual prices/promotions for the full forecast period by setting `period_end=FORECAST_START + FORECAST_HORIZON` above, you can just set `prices_projected_df = prices_raw_df` before moving on.

We can now delete temp variables no longer needed to save memory and make sure we don't accidentally use the wrong ones:

In [ ]:
del prices_future
del prices_raw_df

#### Pulling the RTS together

With our end dates aligned and set up to fully cover the forecast period, we're ready to combine the two datasets and normalize the dimensions to match the Target Time-Series.

First, we'll join them together:

In [ ]:
rts_df = pd.merge(
    holiday_raw_df,
    prices_projected_df,
    on=["date", "country"],
    how="outer",
)

del holiday_raw_df
del prices_projected_df

rts_df

Then expand from `country` to cover all separate `location` IDs and from `product` to cover all separate SKUs. We can refer to the unique location/item_id list saved from the TTS earlier to do this:

In [ ]:
# Join to map country/product to locations/item_ids:
rts_df = (
    pd.merge(
        item_location_combos,
        rts_df,
        on=["country", "product"],
        how="outer",
    )
    .drop(columns=["country", "product"])
)

del item_location_combos

# Standardize timestamp representation, as with TTS:
#rts_df["timestamp"] = rts_df["timestamp"].dt.strftime("%Y-%m-%d")

# Index on key fields:
rts_df = rts_df.set_index(["date", "sku"] + FORECAST_DIMENSIONS)

rts_df

### The combined SageMaker Canvas dataset

Now we have our target (sales) data covering the history up to 2019-11-30, and our related input data covering the history *and forecast horizon* up to 2019-12-31.

While Amazon Forecast treats these as [two separate datasets](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-datasets-groups.html), for Amazon SageMaker Canvas we'll want to combine them together into one:

In [ ]:
%%time
canvas_df = tts_df.join(rts_df, how="outer")

del tts_df
del rts_df

canvas_df

You'll notice some **missing values** in the `demand` column after this join:

1. Missing values **on or after** the `FORECAST_START` date are fine and expected: These will tell Canvas that it needs to forecast for this period but has the other features (holidays and prices) available to help.
2. Missing values **before** the `FORECAST_START` date should be filled with zeros: These emerge because the source sales data is sparse - there simply won't be a record if no units of a product were sold for a particular date/SKU/location/etc combination.

There should not be any missing values in the other columns. We can run some diagnostics here to check everything is as expected before filling:

In [ ]:
print(
    "Last date with historical sales:",
    canvas_df.index.get_level_values("date")[~canvas_df["sales"].isna()].max(),
    "(should be one day before:",
    FORECAST_START,
    ")"
)

print("Missing value counts by column:")
canvas_df.isna().sum()

Then fill all empty sales figures **before** the `FORECAST_START` with 0:

In [ ]:
canvas_df.loc[
    (
        slice(None, FORECAST_START - FORECAST_HORIZON.base),  # (End is inclusive so minus one day)
        canvas_df["sales"].isna(),
    ),
    "sales",
] = 0
canvas_df

This dataset is now ready to use with SageMaker Canvas forecasting. We'll upload it to Amazon S3 directly to use in the tool:

In [ ]:
canvas_s3_uri = f"s3://{BUCKET_NAME}/{BUCKET_PREFIX}training-data/canvas/benefits_demo_canvas.csv"
canvas_df.to_csv(canvas_s3_uri)
print(f"Uploaded SageMaker Canvas dataset to: {canvas_s3_uri}")

## Import your dataset to SageMaker Canvas

1. **If** you ran through the optional data preparation steps above, take note of the final `s3://...` URI of your uploaded dataset.
1. **Otherwise, you can either**:
    1. **Run the code cell below** to transfer the public dataset into your AWS account's Amazon S3 bucket, or
    1. **Download** the (>200MiB) [pre-prepared data from this link](https://measuring-forecast-benefits-assets.s3.amazonaws.com/dataset/v1/benefits_demo_canvas.csv) to your computer, to upload into Canvas later

Since SageMaker notebooks run directly in the AWS Region you deploy them, transferring this large dataset via the notebook may be faster than downloading to your computer. As mentioned up in the [Pre-requisites section](#prereqs) at the beginning of this notebook, you'll need to enable additional permissions in SageMaker Canvas for users to upload local files, if you haven't already.

Either way, since the transfer may take a few minutes, you can go ahead and launch SageMaker Canvas (steps below) while it runs.

In [ ]:
import sagemaker

canvas_s3_uri = "s3://{}/{}training-data/canvas/benefits_demo_canvas.csv".format(
    sagemaker.Session().default_bucket(),  # S3 bucket name as per BUCKET_NAME above
    "measuring-forecast-benefits/",  # S3 folder prefix as per BUCKET_PREFIX above
)

!aws s3 cp s3://measuring-forecast-benefits-assets/dataset/v1/benefits_demo_canvas.csv {canvas_s3_uri}

print(f"Uploaded SageMaker Canvas dataset to: {canvas_s3_uri}")

The way you Canvas will vary depending on how your SageMaker Domain authentication is set up: See the [getting started guide](https://docs.aws.amazon.com/sagemaker/latest/dg/canvas-getting-started.html) for details. For an IAM-authenticated domain as created through "Quick setup", you'll typically launch Canvas from the [Amazon SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/canvas-landing) page for Canvas or for your Domain:

![](img/canvas-01-launch.png "Screenshot of SageMaker Domain users page with option to launch SageMaker Canvas")

> ℹ️ **Tip:** Launching the Canvas UI for a user starts a "session", which may take a minute or two and (as detailed on the [SageMaker Canvas pricing page](https://aws.amazon.com/sagemaker/canvas/pricing/)) is chargeable for the time it's running. Simply closing your browser tab will not shut down the session: Remember to click "log out" in the Canvas UI when you're done, and consider setting up [an automatic shutdown solution](https://aws.amazon.com/blogs/machine-learning/save-costs-by-automatically-shutting-down-idle-resources-within-amazon-sagemaker-studio/) if you manage a domain with several users. You can check the running "apps" for your Studio user at any time by clicking the blue user names in the screen above, to see whether a Canvas session is open.

In the Canvas UI, navigate to **Datasets** in the sidebar menu. You'll usually see a set of sample datasets pre-loaded in a new domain.

Click the **➕ Import** button as shown below to import your own dataset.

![](img/canvas-02-datasets.png "Screenshot of SageMaker Canvas UI Datasets page showing Import button")

If your data is already uploaded to your Amazon S3 account from the code steps above, browse to and select it as shown below. If you instead downloaded the file to your computer, select "Upload" and then drag+drop the file from your computer to upload it.

Once your file is uploaded or selected, click **Import data** to continue.

![](img/canvas-03-import.png "Screenshot of Canvas UI showing target data selected in Amazon S3, with alternative Upload tab also visible")

Once you import, Canvas will save your data and analyze the shape - the processing for which might take a few seconds.

> ⚠️ **Understanding costs:** As well as active sessions, [SageMaker Canvas Pricing](https://aws.amazon.com/sagemaker/canvas/pricing/) charges for model training based on the number of cells in your dataset. The metrics on this dataset list are useful for understanding the cost before training models on your data.
>
> Note that the dataset used in this example could incur significant charges for model training: Around $570 by our estimate at the time of pricing.

## Configure and train your model

When the dataset shows as "Ready", you'll be able to select it and click **Create a model** to start building your model.

![](img/canvas-04-select-data.png "Screenshot selecting data from datasets list in Canvas, showing button to Create a model")

You'll be asked to **enter a name** for your model, and can select one as you like. In our example we used `benefits_demo_canvas`.

On the following screen, you'll see a preview of your data and need to configure your model:

- Set **Target column** to `sales`
- Your **Model type** should be automatically detected as **Time series forecasting**. If not, you can use the "Change type" button to override this.

Note that from this screen you can toggle between (list of) columns view and "grid" view in the dataset preview, and also access other basic data analysis and preparation options. We won't need them here, as our dataset is already prepared:

![](img/canvas-05-model-type.png "Screenshot of Canvas model build with target column selected and time-series model type")

Click **Configure time series model** and, as shown in the screenshot below:

- For **Item ID column** select `sku`
- For **Group column** select `location`
- For **Time stamp column** select `date`
- For **Number of days to forecast** enter `31`
- Leave other options as default

![](img/canvas-06-configure-model.png "Screenshot configuring the forecasting model key fields and horizon")

With these options saved, you may be prompted to **Validate your data**: Accept this for Canvas to check your dataset matches data quality expectations.

Once your data is validated, you should be able to click **Standard build** to start your model training. Note that for some datasets you'll have an alternative "Quick build" option, but at the time of writing this dataset doesn't support that feature:

![](img/canvas-07-start-build.png "Screenshot showing enabled Standard Build button to start model training")

> ⏰ In our experience, a standard build model for this sample dataset will normally take 4-5 hours to train once you start the process.

This is similar to the training time for Amazon Forecast, and in fact if you head over to the [Amazon Forecast console](https://console.aws.amazon.com/forecast/home?#datasetGroups) you'll see that Canvas even uses Amazon Forecast under the hood!

## Review and export forecast results

Once the model is ready, you'll see accuracy metrics and column impact scores on the *Analyze* tab - which can be useful for understanding performance of the model at a high level and which additional columns were most useful for prediction:

![](img/canvas-08-metrics.png "Screenshot of model metrics and feature importance scores")

Over in the **Predict** tab, you can optionally explore predictions for **single** SKU/location combinations in the dataset as shown below. Note that the model gives probabilistic forecasts, with upper-bound and lower-bound confidence estimates as well as the expected value:

![](img/canvas-09-predict-single.png "Screenshot of single-item prediction in SageMaker Canvas showing higher-bound, lower-bound and median quantiles")

To analyze and compare results in the next notebook though, you'll need to select **All items** and then click **Start Predictions** to start building a combined forecast for all products and locations.

> ⏰ Once you start the job, it will take Canvas a few minutes to generate the full set of predictions.

▶️ **Download** your batch predictions from the Canvas UI (about 10MiB in our test), once they're ready, as shown below:

![](img/canvas-10-predict-batch.png "Screenshot of Canvas batch predictions with job complete and results ready to download")

## Next steps

You should now have successfully trained a SageMaker Canvas forecasting model to predict sales in the final month of the sample dataset (2019-12) - and downloaded the results to a local CSV file.

Next, head on over to notebook [2. Measuring Forecast Benefits.ipynb](2.%20Measuring%20Forecast%20Benefits.ipynb) - where we'll compare this forecast to the moving average baseline in terms of the actual business results of applying them in practice.